In [1]:
import os
import numpy as np
import pygmmis

In [12]:
def get_eval_points(samples):
    return np.mgrid[np.min(samples[:, 0]):np.max(samples[:, 0]):12j, 
        np.min(samples[:, 1]):np.max(samples[:, 1]):12j,
        np.min(samples[:, 2]):np.max(samples[:, 2]):12j,
        np.min(samples[:, 3]):np.max(samples[:, 3]):12j,
        np.min(samples[:, 4]):np.max(samples[:, 4]):12j,
        np.min(samples[:, 5]):np.max(samples[:, 5]):12j,
        np.min(samples[:, 6]):np.max(samples[:, 6]):12j].T.reshape(-1, 7)

def score_samples(gmm, points):
      return gmm.logL(points)[0].reshape(12, 12, 12, 12, 12, 12, 12)
    
def test_pgmm_on_chain(objid, imgid, chain, burnin_ratio=2/3, K=30, max_iter=500, tol=5e-4, nthreads=-1, 
                       split_n_merge=False, oversampling=1, cutoff=None, init_method='random'):    
    BURNIN_DISCARD = int(chain.shape[1] * burnin_ratio)
    
    chain = chain[:, BURNIN_DISCARD:, :].reshape(-1, 7)
    
    pgmm = pygmmis.GMM(K=K, D=7)
    
    mins = np.min(chain, axis=0)
    maxs = np.max(chain, axis=0)
    def selcallback(smpls):
        return np.all(np.all([smpls <= maxs, smpls >= mins], axis=0), axis=1)
    
    logL, U = pygmmis.fit(pgmm, chain, sel_callback=selcallback, maxiter=max_iter, tol=tol, init_method=init_method,
                                    oversampling=oversampling, split_n_merge=split_n_merge, cutoff=cutoff) 
    
    pts = get_eval_points(chain)
    
    scores = score_samples(pgmm, pts, nthreads=nthreads, nchunks=-1)

    return pgmm, pts, scores, rmse

In [13]:
chains3 = np.load('chain.npy')
pgmm3, pts3, scores3, rmse3 = test_pgmm_on_chain(objid=3, imgid=3, chain=chains3, K=20, max_iter=500, tol=5e-4, nthreads=-1, 
                                         init_method='kmeans', oversampling=1, cutoff=None, split_n_merge=False)

/root/anaconda3/envs/multiproc/lib/python3.8/site-packages/pygmmis.py:156: RuntimeWarning: invalid value encountered in add
  return np.log(np.exp(logX + c[tuple(c_shape)]).sum(axis=axis)) - c
/root/anaconda3/envs/multiproc/lib/python3.8/site-packages/pygmmis.py:156: RuntimeWarning: invalid value encountered in add
  return np.log(np.exp(logX + c[tuple(c_shape)]).sum(axis=axis)) - c
/root/anaconda3/envs/multiproc/lib/python3.8/site-packages/pygmmis.py:156: RuntimeWarning: invalid value encountered in add
  return np.log(np.exp(logX + c[tuple(c_shape)]).sum(axis=axis)) - c
/root/anaconda3/envs/multiproc/lib/python3.8/site-packages/pygmmis.py:156: RuntimeWarning: invalid value encountered in add
  return np.log(np.exp(logX + c[tuple(c_shape)]).sum(axis=axis)) - c
/root/anaconda3/envs/multiproc/lib/python3.8/site-packages/pygmmis.py:156: RuntimeWarning: invalid value encountered in add
  return np.log(np.exp(logX + c[tuple(c_shape)]).sum(axis=axis)) - c
/root/anaconda3/envs/multiproc/lib/

ValueError: probabilities contain NaN